In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

In [3]:
!aws s3 ls $training_data_uri/

2019-01-24 23:36:40   31360128 eval_data.npy
2019-01-24 23:36:40      40128 eval_labels.npy
2019-01-24 23:36:40  172480128 train_data.npy
2019-01-24 23:36:40     220128 train_labels.npy


In [7]:
! wget --no-check-certificate https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/refs/heads/main/Section-15-Custom-models/mnist-tf2.py

--2025-06-02 11:35:02--  https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/refs/heads/main/Section-15-Custom-models/mnist-tf2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 2308 (2.3K) [text/plain]
Saving to: ‘mnist-tf2.py’

mnist-tf2.py        100%[===================>]   2.25K  --.-KB/s    in 0s      

2025-06-02 11:35:02 (46.2 MB/s) - ‘mnist-tf2.py’ saved [2308/2308]



In [8]:
# TensorFlow 2.x script
!pygmentize 'mnist-tf2.py'

import tensorflow as tf
import argparse
import os
import numpy as np
import json


def model(x_train, y_train, x_test, y_test):
    """Generate a simple model"""
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train)
    model.evaluate(x_test, y_test)

    return model


def _load_training_data(base_dir):
    """Load MNIST training data"""
    x_train = np.load(os.path.join(base_dir, 'train_data.npy'))
    y_train = np.load(os.path.join(base_dir, 'train_labels.npy'))
    return x_train, y_train


def _load_testing_data(base_dir):
    """Load MNIST testing data"""
    x_test = np.load(os.path.join(base_dir, 'eval_data.npy'))
    y_test = np

In [9]:
from sagemaker.tensorflow import TensorFlow
mnist_estimator = TensorFlow(entry_point='mnist-tf2.py',
                             role=role,
                             instance_count=2,
                             instance_type='ml.m5.large',
                             framework_version='2.1.0',
                             py_version='py3',
                             distribution={'parameter_server': {'enabled': True}})

In [10]:
mnist_estimator.fit(training_data_uri)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2025-06-02-11-42-02-498


2025-06-02 11:42:05 Starting - Starting the training job...
2025-06-02 11:42:20 Starting - Preparing the instances for training...
2025-06-02 11:42:44 Downloading - Downloading input data.....2025-06-02 11:43:49,935 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2025-06-02 11:43:49,944 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2025-06-02 11:43:50,050 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2025-06-02 11:43:50,050 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2025-06-02 11:43:50,050 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2025-06-02 11:43:50,083 sagemaker_tensorflow_container.training INFO     Launching worker process
2025-06-02 11:43:50,294 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2025-06-02 11:43:50,332 sagem

In [11]:
mnist_estimator.fit(training_data_uri)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2025-06-02-11-51-03-392


2025-06-02 11:51:08 Starting - Starting the training job...
2025-06-02 11:51:23 Starting - Preparing the instances for training...
2025-06-02 11:51:48 Downloading - Downloading input data......
2025-06-02 11:52:48 Training - Training image download completed. Training in progress.2025-06-02 11:52:51,545 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2025-06-02 11:52:51,554 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2025-06-02 11:52:51,655 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2025-06-02 11:52:51,655 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2025-06-02 11:52:51,655 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2025-06-02 11:52:51,691 sagemaker_tensorflow_container.training INFO     Launching worker process
2025-06-02 11:52:51,926 sagemaker-contain

In [12]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-training-2025-06-02-11-55-24-391
INFO:sagemaker:Creating endpoint-config with name tensorflow-training-2025-06-02-11-55-24-391
INFO:sagemaker:Creating endpoint with name tensorflow-training-2025-06-02-11-55-24-391


---!

In [13]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to ./train_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to ./train_labels.npy


In [15]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

prediction is 7, label is 7, matched: True
prediction is 3, label is 3, matched: True
prediction is 4, label is 4, matched: True
prediction is 6, label is 6, matched: True
prediction is 1, label is 1, matched: True
prediction is 8, label is 8, matched: True
prediction is 1, label is 1, matched: True
prediction is 0, label is 0, matched: True
prediction is 9, label is 9, matched: True
prediction is 8, label is 8, matched: True
prediction is 0, label is 0, matched: True
prediction is 3, label is 3, matched: True
prediction is 1, label is 1, matched: True
prediction is 2, label is 2, matched: True
prediction is 7, label is 7, matched: True
prediction is 0, label is 0, matched: True
prediction is 2, label is 2, matched: True
prediction is 9, label is 9, matched: True
prediction is 6, label is 6, matched: True
prediction is 0, label is 0, matched: True
prediction is 1, label is 1, matched: True
prediction is 6, label is 6, matched: True
prediction is 7, label is 7, matched: True
prediction 

In [ ]:
predictor.delete_endpoint()